In [7]:
#import libraries as needed
!pip install pymongo
from pymongo import MongoClient
from pymongo import MongoClient
from datetime import datetime
import pandas as pd

In [6]:
#upgrade of pip
import sys
!{sys.executable} -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 3.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.8 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [8]:
#Initializing the database
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [48]:
#user document schema

users_schema = {
    "_id": "ObjectId (auto-generated)",

    "firstName": "string (required)",
    "lastName": "string (required)",
    "email": "string (required)",

    "role": "string (enum: ['student', 'instructor'])",

    "dateJoined": "datetime (optional)",

    "profile": {
        "bio": "string (optional)",
        "avatar": "string (optional)",
        "skills": ["string"]  # optional array of strings
    },

    "isActive": "boolean (optional)"
}


In [49]:
users_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["firstName", "lastName", "email", "role"],
        "properties": {
            "firstName": {"bsonType": "string"},
            "lastName": {"bsonType": "string"},
            "email": {"bsonType": "string"},
            "role": {"enum": ["student", "instructor"]},
            "dateJoined": {"bsonType": "date"},
            "profile": {
                "bsonType": "object",
                "properties": {
                    "bio": {"bsonType": "string"},
                    "avatar": {"bsonType": "string"},
                    "skills": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    }
                }
            },
            "isActive": {"bsonType": "bool"}
        }
    }
}

In [24]:
course_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["courseId", "title", "instructorId", "level", "duration", "price", "createdAt", "updatedAt", "isPublished"],
        "properties": {
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "instructorId": {"bsonType": "string"},  # referencing user.userId
            "category": {"bsonType": "string"},
            "level": {"enum": ["beginner", "intermediate", "advanced"]},
            "duration": {"bsonType": ["double", "int"]},  # to handle both 8.2 and 8
            "price": {"bsonType": ["double", "int"]},
            "tags": {
                "bsonType": "array",
                "items": {"bsonType": "string"}
            },
            "createdAt": {"bsonType": "date"},
            "updatedAt": {"bsonType": "date"},
            "isPublished": {"bsonType": "bool"}
        }
    }
}

# Apply this validator to the 'courses' collection
db.command({
    "collMod": "courses",
    "validator": course_validator,
    "validationLevel": "moderate"
})

{'ok': 1.0}

In [14]:
course_schema = {

    "_id": "ObjectId (auto-generated)",

    "courseId": "string (unique)",

    "title": "string (required)",

    "description": "string",

    "instructorId": "string (reference to users)",

    "category": "string",

    "level": "string (enum: [‘beginner’, ‘intermediate’, ‘advanced’])",

    "duration": "number (in hours)",

    "price": "number",

    "tags": ["string"],

    "createdAt": "datetime",

    "updatedAt": "datetime",

    "isPublished": "boolean"

}

In [27]:
# Enrollments collection
enrollment_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["enrollmentId", "studentId", "courseId", "enrolledAt", "progress", "status"],
        "properties": {
            "enrollmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},  # Should match users.userId
            "courseId": {"bsonType": "string"},   # Should match courses.courseId
            "enrolledAt": {"bsonType": "date"},
            "progress": {
                "bsonType": ["int", "double"],
                "minimum": 0,
                "maximum": 100
            },
            "status": {
                "enum": ["enrolled", "completed", "dropped"]
            }
        }
    }
}

# Apply the validator to the enrollments collection
db.command({
    "collMod": "enrollments",
    "validator": enrollment_validator,
    "validationLevel": "moderate"
})


{'ok': 1.0}

In [ ]:
enrollment_schema = {
    "_id": "ObjectId (auto-generated)",

    "enrollmentId": "string (unique)",

    "studentId": "string (reference to users.userId)",

    "courseId": "string (reference to courses.courseId)",

    "enrolledAt": "datetime",

    "progress": "number (percentage, e.g., 75)",

    "status": "string (enum: ['enrolled', 'completed', 'dropped'])"
}

In [29]:
lesson_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["lessonId", "courseId", "title", "content", "position", "createdAt", "updatedAt"],
        "properties": {
            "lessonId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},  # Reference to courses.courseId
            "title": {"bsonType": "string"},
            "content": {"bsonType": "string"},   # Can contain HTML or markdown
            "videoUrl": {
                "bsonType": "string"
            },
            "position": {
                "bsonType": ["int", "double"],
                "minimum": 1
            },
            "createdAt": {"bsonType": "date"},
            "updatedAt": {"bsonType": "date"}
        }
    }
}

# Apply the validator to the lessons collection
db.command({
    "collMod": "lessons",
    "validator": lesson_validator,
    "validationLevel": "moderate"
})


{'ok': 1.0}

In [ ]:
lesson_schema = {
    "_id": "ObjectId (auto-generated)",

    "lessonId": "string (unique)",

    "courseId": "string (reference to courses.courseId)",

    "title": "string (required)",

    "content": "string (HTML or markdown)",

    "videoUrl": "string (optional)",

    "position": "number (order in course)",

    "createdAt": "datetime",

    "updatedAt": "datetime"
}

In [31]:
#Assignment validator
assignment_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["assignmentId", "courseId", "title", "dueDate", "totalMarks", "createdAt", "updatedAt"],
        "properties": {
            "assignmentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},  # Reference to courses.courseId
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "dueDate": {"bsonType": "date"},
            "totalMarks": {
                "bsonType": ["int", "double"],
                "minimum": 0
            },
            "createdAt": {"bsonType": "date"},
            "updatedAt": {"bsonType": "date"}
        }
    }
}

# Apply the validator to the assignments collection
db.command({
    "collMod": "assignments",
    "validator": assignment_validator,
    "validationLevel": "moderate"
})

{'ok': 1.0}

In [ ]:
assignment_schema = {
    "_id": "ObjectId (auto-generated)",

    "assignmentId": "string (unique)",

    "courseId": "string (reference to courses.courseId)",

    "title": "string (required)",

    "description": "string",

    "dueDate": "datetime",

    "totalMarks": "number",

    "createdAt": "datetime",

    "updatedAt": "datetime"
}

In [35]:
#Submission validator
submission_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": [
            "submissionId",
            "assignmentId",
            "studentId",
            "submittedAt",
            "content",
            "status"
        ],
        "properties": {
            "submissionId": {"bsonType": "string"},
            "assignmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "submittedAt": {"bsonType": "date"},
            "content": {"bsonType": "string"},
            "grade": {
                "bsonType": "object",
                "properties": {
                    "score": {
                        "bsonType": ["int", "double"],
                        "minimum": 0
                    },
                    "feedback": {"bsonType": "string"},
                    "gradedAt": {"bsonType": "date"},
                    "gradedBy": {"bsonType": "string"}
                }
            },
            "status": {
                "enum": ["submitted", "graded", "late"]
            }
        }
    }
}


In [ ]:
submission_schema = {
    "_id": "ObjectId (auto-generated)",

    "submissionId": "string (unique)",

    "assignmentId": "string (reference to assignments.assignmentId)",

    "studentId": "string (reference to users.userId)",

    "submittedAt": "datetime",

    "content": "string (URL, text, or file reference)",

    "grade": {
        "score": "number",
        "feedback": "string",
        "gradedAt": "datetime",
        "gradedBy": "string (instructorId)"
    },

    "status": "string (enum: ['submitted', 'graded', 'late'])"
}

In [16]:
import random
import uuid

In [20]:
db.command({
    "collMod": "users",
    "validator": user_validator,
    "validationLevel": "moderate"
})

{'ok': 1.0}

In [122]:
import pandas as pd

In [ ]:
#insert users
from datetime import datetime
import random
import uuid

roles = ['student', 'instructor']
users = []

for i in range(20):
    role = random.choice(roles)
    user = {
        "userId": str(uuid.uuid4()),
        "email": f"user{i}@eduhub.com",
        "firstName": f"First{i}",
        "lastName": f"Last{i}",
        "role": role,
        "dateJoined": datetime.now(),
        "profile": {
            "bio": f"This is user {i}'s bio.",
            "avatar": f"https://avatar.com/u{i}.png",
            "skills": ["Python", "Data", "Web"][0:random.randint(1, 3)]
        },
        "isActive": True
    }
    users.append(user)

# Insert into MongoDB
db.users.insert_many(users)

In [25]:
#Insert Courses

course_levels = ['beginner', 'intermediate', 'advanced']
categories = ['Data Science', 'Web Dev', 'AI', 'Cloud']
instructors = [u for u in users if u['role'] == 'instructor']
courses = []

for i in range(8):
    instructor = random.choice(instructors)
    course = {
        "courseId": str(uuid.uuid4()),
        "title": f"Course {i}",
        "description": f"This is course {i} description.",
        "instructorId": instructor["userId"],
        "category": random.choice(categories),
        "level": random.choice(course_levels),
        "duration": round(random.uniform(1.5, 10), 1),
        "price": round(random.uniform(10, 200), 2),
        "tags": ["tech", "edu", "skill"][0:random.randint(1, 3)],
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": True
    }
    courses.append(course)

db.courses.insert_many(courses)


InsertManyResult([ObjectId('684fbc191e9483271544a5df'), ObjectId('684fbc191e9483271544a5e0'), ObjectId('684fbc191e9483271544a5e1'), ObjectId('684fbc191e9483271544a5e2'), ObjectId('684fbc191e9483271544a5e3'), ObjectId('684fbc191e9483271544a5e4'), ObjectId('684fbc191e9483271544a5e5'), ObjectId('684fbc191e9483271544a5e6')], acknowledged=True)

In [28]:
#Insert enrollments
students = [u for u in users if u['role'] == 'student']
enrollments = []

for i in range(15):
    enrollment = {
        "enrollmentId": str(uuid.uuid4()),
        "studentId": random.choice(students)["userId"],
        "courseId": random.choice(courses)["courseId"],
        "enrolledAt": datetime.now(),
        "progress": random.randint(0, 100),
        "status": random.choice(['enrolled', 'completed', 'dropped'])
    }
    enrollments.append(enrollment)

db.enrollments.insert_many(enrollments)

InsertManyResult([ObjectId('684fbe431e9483271544a5f6'), ObjectId('684fbe431e9483271544a5f7'), ObjectId('684fbe431e9483271544a5f8'), ObjectId('684fbe431e9483271544a5f9'), ObjectId('684fbe431e9483271544a5fa'), ObjectId('684fbe431e9483271544a5fb'), ObjectId('684fbe431e9483271544a5fc'), ObjectId('684fbe431e9483271544a5fd'), ObjectId('684fbe431e9483271544a5fe'), ObjectId('684fbe431e9483271544a5ff'), ObjectId('684fbe431e9483271544a600'), ObjectId('684fbe431e9483271544a601'), ObjectId('684fbe431e9483271544a602'), ObjectId('684fbe431e9483271544a603'), ObjectId('684fbe431e9483271544a604')], acknowledged=True)

In [30]:
#insert Lesson
lessons = []

for i in range(25):
    course = random.choice(courses)
    lesson = {
        "lessonId": str(uuid.uuid4()),
        "courseId": course["courseId"],
        "title": f"Lesson {i}",
        "content": f"Lesson {i} content in markdown or HTML.",
        "videoUrl": f"https://videos.com/lesson{i}",
        "position": random.randint(1, 10),
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }
    lessons.append(lesson)

db.lessons.insert_many(lessons)

InsertManyResult([ObjectId('684fbf781e9483271544a605'), ObjectId('684fbf781e9483271544a606'), ObjectId('684fbf781e9483271544a607'), ObjectId('684fbf781e9483271544a608'), ObjectId('684fbf781e9483271544a609'), ObjectId('684fbf781e9483271544a60a'), ObjectId('684fbf781e9483271544a60b'), ObjectId('684fbf781e9483271544a60c'), ObjectId('684fbf781e9483271544a60d'), ObjectId('684fbf781e9483271544a60e'), ObjectId('684fbf781e9483271544a60f'), ObjectId('684fbf781e9483271544a610'), ObjectId('684fbf781e9483271544a611'), ObjectId('684fbf781e9483271544a612'), ObjectId('684fbf781e9483271544a613'), ObjectId('684fbf781e9483271544a614'), ObjectId('684fbf781e9483271544a615'), ObjectId('684fbf781e9483271544a616'), ObjectId('684fbf781e9483271544a617'), ObjectId('684fbf781e9483271544a618'), ObjectId('684fbf781e9483271544a619'), ObjectId('684fbf781e9483271544a61a'), ObjectId('684fbf781e9483271544a61b'), ObjectId('684fbf781e9483271544a61c'), ObjectId('684fbf781e9483271544a61d')], acknowledged=True)

In [32]:
#Insert assignment
assignments = []

for i in range(10):
    course = random.choice(courses)
    assignment = {
        "assignmentId": str(uuid.uuid4()),
        "courseId": course["courseId"],
        "title": f"Assignment {i}",
        "description": f"This is the description for assignment {i}.",
        "dueDate": datetime(2025, 12, 31),
        "totalMarks": 100,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }
    assignments.append(assignment)

db.assignments.insert_many(assignments)


InsertManyResult([ObjectId('684fc05f1e9483271544a61e'), ObjectId('684fc05f1e9483271544a61f'), ObjectId('684fc05f1e9483271544a620'), ObjectId('684fc05f1e9483271544a621'), ObjectId('684fc05f1e9483271544a622'), ObjectId('684fc05f1e9483271544a623'), ObjectId('684fc05f1e9483271544a624'), ObjectId('684fc05f1e9483271544a625'), ObjectId('684fc05f1e9483271544a626'), ObjectId('684fc05f1e9483271544a627')], acknowledged=True)

In [36]:
#insert submission
submissions = []

for i in range(12):
    assignment = random.choice(assignments)
    student = random.choice(students)
    graded = random.choice([True, False])

    submission = {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": assignment["assignmentId"],
        "studentId": student["userId"],
        "submittedAt": datetime.now(),
        "content": f"https://submissions.com/s{i}",
        "status": "graded" if graded else "submitted"
    }

    if graded:
        submission["grade"] = {
            "score": random.randint(0, 100),
            "feedback": "Good job!",
            "gradedAt": datetime.now(),
            "gradedBy": random.choice(instructors)["userId"]
        }

    submissions.append(submission)

db.submissions.insert_many(submissions)


InsertManyResult([ObjectId('684fc3301e9483271544a634'), ObjectId('684fc3301e9483271544a635'), ObjectId('684fc3301e9483271544a636'), ObjectId('684fc3301e9483271544a637'), ObjectId('684fc3301e9483271544a638'), ObjectId('684fc3301e9483271544a639'), ObjectId('684fc3301e9483271544a63a'), ObjectId('684fc3301e9483271544a63b'), ObjectId('684fc3301e9483271544a63c'), ObjectId('684fc3301e9483271544a63d'), ObjectId('684fc3301e9483271544a63e'), ObjectId('684fc3301e9483271544a63f')], acknowledged=True)

In [39]:
# 3. Create collections with validation rules
def create_or_update_collection(name, validator):
    if name in db.list_collection_names():
        # Collection exists: update validator
        db.command({
            "collMod": name,
            "validator": validator,
            "validationLevel": "moderate"  # or "strict" if you prefer
        })
        print(f"Updated validator for existing collection: {name}")
    else:
        # Create new collection with validator
        db.create_collection(name, validator=validator)
        print(f"Created new collection with validator: {name}")


# Apply to all your collections
create_or_update_collection("user", user_validator)
create_or_update_collection("courses", course_validator)
create_or_update_collection("enrollments", enrollment_validator)
create_or_update_collection("lessons", lesson_validator)
create_or_update_collection("assignments", assignment_validator)
create_or_update_collection("submissions", submission_validator)

print("All collections created or updated successfully.")


Created new collection with validator: user
Updated validator for existing collection: courses
Updated validator for existing collection: enrollments
Updated validator for existing collection: lessons
Updated validator for existing collection: assignments
Updated validator for existing collection: submissions
All collections created or updated successfully.


In [42]:
collections = db.list_collection_names()

print("Collections in 'eduhub_db':")
for col in collections:
    print(f" - {col}")

Collections in 'eduhub_db':
 - users
 - courses
 - user
 - assignments
 - submissions
 - enrollments
 - lessons


In [46]:
# Delete 'user' collection if it exists
if "user" in db.list_collection_names():
    db.user.drop()
    print("Dropped 'user' collection.")

# Delete 'users' collection if it exists
if "users" in db.list_collection_names():
    db.users.drop()
    print("Dropped 'users' collection.")


Dropped 'user' collection.
Dropped 'users' collection.


In [51]:
collections = db.list_collection_names()

print("Collections in 'eduhub_db':")
for col in collections:
    print(f" - {col}")

Collections in 'eduhub_db':
 - courses
 - assignments
 - submissions
 - users
 - enrollments
 - lessons


In [55]:
from pprint import pprint

# Retrieve all documents from the 'users' collection
users = db.users.find()

# Print each user document
print("Documents in 'users' collection:")
for user in users:
    pprint(user)

Documents in 'users' collection:
{'_id': ObjectId('684fcbd81e9483271544a641'),
 'dateJoined': datetime.datetime(2025, 6, 16, 1, 46, 32, 410000),
 'email': 'user0@eduhub.com',
 'firstName': 'First0',
 'isActive': True,
 'lastName': 'Last0',
 'profile': {'avatar': 'https://avatar.com/u0.png',
             'bio': "This is user 0's bio.",
             'skills': ['Python', 'Data', 'Web']},
 'role': 'instructor',
 'userId': '363b3454-b7e4-430b-813b-bd84fe263cb8'}
{'_id': ObjectId('684fcbd81e9483271544a642'),
 'dateJoined': datetime.datetime(2025, 6, 16, 1, 46, 32, 410000),
 'email': 'user1@eduhub.com',
 'firstName': 'First1',
 'isActive': True,
 'lastName': 'Last1',
 'profile': {'avatar': 'https://avatar.com/u1.png',
             'bio': "This is user 1's bio.",
             'skills': ['Python']},
 'role': 'student',
 'userId': 'bc6dd3a0-86bf-44d1-8940-2b5ca5caefd4'}
{'_id': ObjectId('684fcbd81e9483271544a643'),
 'dateJoined': datetime.datetime(2025, 6, 16, 1, 46, 32, 410000),
 'email': 'user

In [56]:
#Basic CRUD Operations, task3.1
# 1. Add a new student user (append to existing users)
new_student = {
    "userId": str(uuid.uuid4()),
    "firstName": "Ada",
    "lastName": "Okoro",
    "email": "ada.okoro@example.com",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "A passionate learner",
        "avatar": "https://example.com/avatar.png",
        "skills": ["Python", "MongoDB"]
    },
    "isActive": True
}
db.users.insert_one(new_student)
print("✅ New student user added.")

✅ New student user added.


In [57]:
# 2. Create a new course (append to existing courses)
# Ensure there’s at least one instructor to assign
instructor = db.users.find_one({"role": "instructor"})
if not instructor:
    raise Exception("No instructor found. Add one first.")

new_course = {
    "courseId": str(uuid.uuid4()),
    "title": "Data Visualization with Python",
    "description": "Learn how to visualize data using matplotlib and seaborn.",
    "instructorId": instructor["userId"],
    "category": "Data Science",
    "level": "intermediate",
    "duration": 5.5,
    "price": 89.99,
    "tags": ["data", "visualization", "python"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}
db.courses.insert_one(new_course)
print("✅ New course added.")

✅ New course added.


In [58]:
# 3. Enroll the new student in the new course (append to enrollments)
new_enrollment = {
    "enrollmentId": str(uuid.uuid4()),
    "studentId": new_student["userId"],
    "courseId": new_course["courseId"],
    "enrolledAt": datetime.now(),
    "progress": 0,
    "status": "enrolled"
}
db.enrollments.insert_one(new_enrollment)
print("✅ Student enrolled in course.")

✅ Student enrolled in course.


In [59]:
# 4. Add a new lesson to the existing course (append to lessons)
new_lesson = {
    "lessonId": str(uuid.uuid4()),
    "courseId": new_course["courseId"],
    "title": "Lesson 1: Introduction to Data Visualization",
    "content": "<p>This lesson covers the basics of data visualization using Python libraries.</p>",
    "videoUrl": "https://videos.example.com/lesson1",
    "position": 1,
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}
db.lessons.insert_one(new_lesson)
print("✅ New lesson added to course.")

✅ New lesson added to course.


In [61]:
import re

In [62]:
#Task 3.2 Read Operations
# 1. Find all active students
active_students = list(db.users.find({
    "role": "student",
    "isActive": True
}))
print("✅ Active students:")
for student in active_students:
    print(f"- {student['firstName']} {student['lastName']}")

# 2. Retrieve course details with instructor information (using aggregation)
courses_with_instructors = list(db.courses.aggregate([
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {"$unwind": "$instructor"}
]))
print("\n✅ Courses with instructor details:")
for course in courses_with_instructors:
    print(f"- {course['title']} by {course['instructor']['firstName']} {course['instructor']['lastName']}")

# 3. Get all courses in a specific category (e.g., "Data Science")
category = "Data Science"
data_science_courses = list(db.courses.find({
    "category": category
}))
print(f"\n✅ Courses in category '{category}':")
for course in data_science_courses:
    print(f"- {course['title']}")

# 4. Find students enrolled in a particular course
# Example: using a specific courseId
course_id = courses_with_instructors[0]["courseId"]  # just taking the first course
enrollments = list(db.enrollments.find({
    "courseId": course_id
}))

student_ids = [e["studentId"] for e in enrollments]
students_in_course = list(db.users.find({
    "userId": {"$in": student_ids}
}))

print(f"\n✅ Students enrolled in course '{courses_with_instructors[0]['title']}':")
for student in students_in_course:
    print(f"- {student['firstName']} {student['lastName']}")

# 5. Search courses by title (case-insensitive, partial match)
search_term = "python"
matched_courses = list(db.courses.find({
    "title": {"$regex": re.compile(search_term, re.IGNORECASE)}
}))
print(f"\n✅ Courses matching '{search_term}' in title:")
for course in matched_courses:
    print(f"- {course['title']}")

✅ Active students:
- First1 Last1
- First3 Last3
- First4 Last4
- First6 Last6
- First10 Last10
- First11 Last11
- First12 Last12
- First14 Last14
- First16 Last16
- First17 Last17
- First18 Last18
- Ada Okoro

✅ Courses with instructor details:
- Data Visualization with Python by First0 Last0

✅ Courses in category 'Data Science':
- Course 3
- Course 5
- Data Visualization with Python

✅ Students enrolled in course 'Data Visualization with Python':
- Ada Okoro

✅ Courses matching 'python' in title:
- Data Visualization with Python


In [63]:
#Task 3.3 Update Operations
db.users.update_one(
    {"email": "user1@eduhub.com"},
    {
        "$set": {
            "profile.bio": "Updated bio for user1.",
            "profile.avatar": "https://new-avatar.com/u1.png"
        }
    }
)


#Mark a course as published
db.courses.update_one(
    {"title": "Course 3"},
    {"$set": {"isPublished": True}}
)


#Add tags to an existing course

db.courses.update_one(
    {"title": "Course 5"},
    {
        "$addToSet": {
            "tags": {"$each": ["project", "practical"]}
        }
    }
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [64]:
#Task3.4 Delete operation
#1 Remove a user
db.users.update_one(
    {"email": "student1@eduhub.com"},
    {"$set": {"isActive": False}}
)

#2 Delete enrollment
db.enrollments.delete_one(
    {"enrollmentId": "your-enrollment-id-here"}
)
#3 Remove lesson from course
db.lessons.delete_one(
    {"lessonId": "your-lesson-id-here"}
)

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [65]:
#Task 4.1.1; complex queries
courses_in_range = list(db.courses.find({
    "price": {"$gte": 50, "$lte": 200}
}))

print("Courses priced between $50 and $200:")
for course in courses_in_range:
    print(course["title"], "-", course["price"])


Courses priced between $50 and $200:
Course 0 - 121.34
Course 1 - 187.85
Course 2 - 91.72
Course 4 - 92.18
Course 5 - 129.14
Course 6 - 69.23
Course 7 - 92.14
Data Visualization with Python - 89.99


In [66]:
#Task 4.1.2 users who joined in the last 6months

from datetime import datetime, timedelta

six_months_ago = datetime.now() - timedelta(days=6*30)  # approximate 6 months
recent_users = list(db.users.find({
    "dateJoined": {"$gte": six_months_ago}
}))

print("\nUsers who joined in the last 6 months:")
for user in recent_users:
    print(user["firstName"], user["lastName"], "-", user["dateJoined"])



Users who joined in the last 6 months:
First0 Last0 - 2025-06-16 01:46:32.410000
First1 Last1 - 2025-06-16 01:46:32.410000
First2 Last2 - 2025-06-16 01:46:32.410000
First3 Last3 - 2025-06-16 01:46:32.410000
First4 Last4 - 2025-06-16 01:46:32.410000
First5 Last5 - 2025-06-16 01:46:32.410000
First6 Last6 - 2025-06-16 01:46:32.410000
First7 Last7 - 2025-06-16 01:46:32.410000
First8 Last8 - 2025-06-16 01:46:32.410000
First9 Last9 - 2025-06-16 01:46:32.410000
First10 Last10 - 2025-06-16 01:46:32.410000
First11 Last11 - 2025-06-16 01:46:32.410000
First12 Last12 - 2025-06-16 01:46:32.410000
First13 Last13 - 2025-06-16 01:46:32.410000
First14 Last14 - 2025-06-16 01:46:32.410000
First15 Last15 - 2025-06-16 01:46:32.410000
First16 Last16 - 2025-06-16 01:46:32.410000
First17 Last17 - 2025-06-16 01:46:32.410000
First18 Last18 - 2025-06-16 01:46:32.410000
First19 Last19 - 2025-06-16 01:46:32.410000
Ada Okoro - 2025-06-16 02:07:03.657000


In [67]:
#Task 4.1.3 Find courses that have specific tags using $in operator
tags_to_find = ["tech", "python", "cloud"]

courses_with_tags = list(db.courses.find({
    "tags": {"$in": tags_to_find}
}))

print("\nCourses with specified tags:")
for course in courses_with_tags:
    print(course["title"], "-", course["tags"])



Courses with specified tags:
Course 0 - ['tech']
Course 1 - ['tech', 'edu']
Course 2 - ['tech']
Course 3 - ['tech', 'edu']
Course 4 - ['tech']
Course 5 - ['tech', 'edu', 'project', 'practical']
Course 6 - ['tech', 'edu', 'skill']
Course 7 - ['tech']
Data Visualization with Python - ['data', 'visualization', 'python']


In [68]:
#Task 4.1.4 Retrieve assignment with due dates in the next week
today = datetime.now()
next_week = today + timedelta(days=7)

upcoming_assignments = list(db.assignments.find({
    "dueDate": {"$gte": today, "$lte": next_week}
}))

print("\nAssignments due in the next 7 days:")
for assignment in upcoming_assignments:
    print(assignment["title"], "-", assignment["dueDate"])



Assignments due in the next 7 days:


In [69]:
#Task 4.2.1 Count total enrollment per course
pipeline_enrollments_per_course = [
    {
        "$group": {
            "_id": "$courseId",
            "totalEnrollments": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "courses",
            "localField": "_id",
            "foreignField": "courseId",
            "as": "courseDetails"
        }
    },
    {
        "$unwind": "$courseDetails"
    },
    {
        "$project": {
            "courseTitle": "$courseDetails.title",
            "totalEnrollments": 1
        }
    }
]

results = db.enrollments.aggregate(pipeline_enrollments_per_course)

print("Total enrollments per course:")
for doc in results:
    print(f"{doc['courseTitle']} → {doc['totalEnrollments']} enrollments")


Total enrollments per course:
Course 4 → 3 enrollments
Course 1 → 2 enrollments
Course 5 → 1 enrollments
Course 3 → 2 enrollments
Course 7 → 4 enrollments
Data Visualization with Python → 1 enrollments
Course 2 → 1 enrollments
Course 6 → 2 enrollments


In [71]:
#Task 4.2.1 Average course rating
pipeline_avg_rating = [
    {
        "$project": {
            "title": 1,
            "avgRating": {"$avg": "$ratings"}
        }
    }
]

results = db.courses.aggregate(pipeline_avg_rating)

print("\n Average rating per course:")
for doc in results:
    print(f"{doc['title']} → {round(doc['avgRating'], 2) if doc['avgRating'] else 'No ratings'}")



 Average rating per course:
Course 0 → No ratings
Course 1 → No ratings
Course 2 → No ratings
Course 3 → No ratings
Course 4 → No ratings
Course 5 → No ratings
Course 6 → No ratings
Course 7 → No ratings
Data Visualization with Python → No ratings


In [72]:
#Task 4.2.1 Group by course category
pipeline_group_by_category = [
    {
        "$group": {
            "_id": "$category",
            "totalCourses": {"$sum": 1}
        }
    },
    {
        "$sort": {"totalCourses": -1}
    }
]

results = db.courses.aggregate(pipeline_group_by_category)

print("\n Total courses by category:")
for doc in results:
    print(f"{doc['_id']} → {doc['totalCourses']} courses")



 Total courses by category:
Data Science → 3 courses
Cloud → 3 courses
Web Dev → 3 courses


In [74]:
#Task 4.2.2 Average grade per student
pipeline_avg_grade_per_student = [
    {
        "$match": {
            "grade.score": {"$ne": None}
        }
    },
    {
        "$group": {
            "_id": "$studentId",
            "averageGrade": {"$avg": "$grade.score"}
        }
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "userId",
            "as": "student"
        }
    },
    {"$unwind": "$student"},
    {
        "$project": {
            "studentName": {
                "$concat": ["$student.firstName", " ", "$student.lastName"]
            },
            "averageGrade": 1
        }
    },
    {"$sort": {"averageGrade": -1}}
]

results = db.submissions.aggregate(pipeline_avg_grade_per_student)

print("\n Average grade per student:")
for doc in results:
    print(f"{doc['studentName']} → {round(doc['averageGrade'], 2)}%")




 Average grade per student:


In [75]:
#Task 4.2.2 Completion rate by course
pipeline_completion_rate = [
    {
        "$group": {
            "_id": "$courseId",
            "total": {"$sum": 1},
            "completed": {
                "$sum": {
                    "$cond": [{"$eq": ["$status", "completed"]}, 1, 0]
                }
            }
        }
    },
    {
        "$project": {
            "completionRate": {
                "$cond": [
                    {"$eq": ["$total", 0]},
                    0,
                    {"$multiply": [{"$divide": ["$completed", "$total"]}, 100]}
                ]
            }
        }
    },
    {
        "$lookup": {
            "from": "courses",
            "localField": "_id",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$project": {
            "courseTitle": "$course.title",
            "completionRate": 1
        }
    },
    {"$sort": {"completionRate": -1}}
]

results = db.enrollments.aggregate(pipeline_completion_rate)

print("\nCompletion rate by course:")
for doc in results:
    print(f"{doc['courseTitle']} → {round(doc['completionRate'], 2)}%")



Completion rate by course:
Course 3 → 100.0%
Course 2 → 100.0%
Course 1 → 50.0%
Course 7 → 50.0%
Course 4 → 0.0%
Course 5 → 0.0%
Data Visualization with Python → 0.0%
Course 6 → 0.0%


In [82]:
#inserting extraa submissions
from datetime import datetime
import uuid
import random

instructors = list(db.users.find({"role": "instructor"}))
students = list(db.users.find({"role": "student"}))
assignments = list(db.assignments.find())

submissions = []

for i in range(12):
    assignment = random.choice(assignments)
    student = random.choice(students)

    # Force all to be graded
    submission = {
        "submissionId": str(uuid.uuid4()),
        "assignmentId": assignment["assignmentId"],
        "studentId": student["userId"],
        "submittedAt": datetime.now(),
        "content": f"https://submissions.com/s{i}",
        "grade": {
            "score": random.randint(60, 100),
            "feedback": "Nice work!",
            "gradedAt": datetime.now(),
            "gradedBy": random.choice(instructors)["userId"]
        },
        "status": "graded"
    }

    submissions.append(submission)

db.submissions.insert_many(submissions)
print("Sample graded submissions inserted.")


Sample graded submissions inserted.


In [84]:
#Task 4.2.2 Top performing Students
from bson.son import SON

pipeline_top_students = [
    { "$match": { "grade.score": { "$ne": None } } },
    { "$group": {
        "_id": "$studentId",
        "averageGrade": { "$avg": "$grade.score" }
    }},
    { "$lookup": {
        "from": "users",
        "localField": "_id",
        "foreignField": "userId",
        "as": "studentInfo"
    }},
    { "$unwind": "$studentInfo" },
    { "$addFields": {
        "studentName": {
            "$concat": ["$studentInfo.firstName", " ", "$studentInfo.lastName"]
        }
    }},
    { "$sort": SON([("averageGrade", -1)]) },
    { "$limit": 5 }
]

results = db.submissions.aggregate(pipeline_top_students)

print("\nTop 5 performing students:")
for doc in results:
    print(f"{doc['studentName']} → {round(doc['averageGrade'], 2)}%")



Top 5 performing students:
First10 Last10 → 88.0%
First12 Last12 → 87.0%
Ada Okoro → 83.67%
First6 Last6 → 77.0%
First18 Last18 → 75.67%


In [85]:
#Task 4.2.3 Total student taught by each instructor
pipeline_total_students_per_instructor = [
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.instructorId",
            "uniqueStudents": {"$addToSet": "$studentId"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "instructorId": "$_id",
            "totalStudents": {"$size": "$uniqueStudents"}
        }
    }
]

results = db.enrollments.aggregate(pipeline_total_students_per_instructor)
print("\nTotal students per instructor:")
for doc in results:
    print(doc)



Total students per instructor:
{'instructorId': '363b3454-b7e4-430b-813b-bd84fe263cb8', 'totalStudents': 1}
{'instructorId': 'd15bd8da-6260-4594-8281-6f7e34c0b971', 'totalStudents': 4}
{'instructorId': '8c2b1b18-94ed-455d-bd7f-3fe54c6b8393', 'totalStudents': 1}
{'instructorId': '5003e27d-dd06-41d4-aa86-e783f5d2cb96', 'totalStudents': 5}
{'instructorId': '25717c0a-40d8-4de1-9fce-85184d6928dc', 'totalStudents': 2}


In [86]:
#Task 4.2.3 Average course rating per instructor
pipeline_avg_rating_per_instructor = [
    {
        "$group": {
            "_id": "$instructorId",
            "averageRating": {"$avg": "$rating"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "instructorId": "$_id",
            "averageRating": {"$round": ["$averageRating", 2]}
        }
    }
]

results = db.courses.aggregate(pipeline_avg_rating_per_instructor)
print("\nAverage course rating per instructor:")
for doc in results:
    print(doc)



Average course rating per instructor:
{'instructorId': '363b3454-b7e4-430b-813b-bd84fe263cb8', 'averageRating': None}
{'instructorId': '5003e27d-dd06-41d4-aa86-e783f5d2cb96', 'averageRating': None}
{'instructorId': '25717c0a-40d8-4de1-9fce-85184d6928dc', 'averageRating': None}
{'instructorId': '8c2b1b18-94ed-455d-bd7f-3fe54c6b8393', 'averageRating': None}
{'instructorId': 'd15bd8da-6260-4594-8281-6f7e34c0b971', 'averageRating': None}


In [87]:
#Task 4.2.3 Revenue generated by instructor
pipeline_revenue_per_instructor = [
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.instructorId",
            "totalRevenue": {"$sum": "$course.price"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "instructorId": "$_id",
            "totalRevenue": {"$round": ["$totalRevenue", 2]}
        }
    }
]

results = db.enrollments.aggregate(pipeline_revenue_per_instructor)
print("\nRevenue per instructor:")
for doc in results:
    print(doc)




Revenue per instructor:
{'instructorId': 'd15bd8da-6260-4594-8281-6f7e34c0b971', 'totalRevenue': 368.56}
{'instructorId': '363b3454-b7e4-430b-813b-bd84fe263cb8', 'totalRevenue': 89.99}
{'instructorId': '5003e27d-dd06-41d4-aa86-e783f5d2cb96', 'totalRevenue': 728.4}
{'instructorId': '25717c0a-40d8-4de1-9fce-85184d6928dc', 'totalRevenue': 230.18}
{'instructorId': '8c2b1b18-94ed-455d-bd7f-3fe54c6b8393', 'totalRevenue': 129.14}


In [88]:
#task 4.2.4 Monthly enrollment trends
pipeline_monthly_enrollments = [
    {
        "$group": {
            "_id": {
                "year": {"$year": "$enrolledAt"},
                "month": {"$month": "$enrolledAt"}
            },
            "enrollmentCount": {"$sum": 1}
        }
    },
    {"$sort": {"_id.year": 1, "_id.month": 1}},
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "enrollmentCount": 1
        }
    }
]

results = db.enrollments.aggregate(pipeline_monthly_enrollments)
print("\nMonthly Enrollment Trends:")
for doc in results:
    print(doc)



Monthly Enrollment Trends:
{'enrollmentCount': 16, 'year': 2025, 'month': 6}


In [89]:
#Task 4.2.4 Most popular course categories
pipeline_popular_categories = [
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {"$unwind": "$course"},
    {
        "$group": {
            "_id": "$course.category",
            "totalEnrollments": {"$sum": 1}
        }
    },
    {"$sort": {"totalEnrollments": -1}},
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "totalEnrollments": 1
        }
    }
]

results = db.enrollments.aggregate(pipeline_popular_categories)
print("\nMost Popular Course Categories:")
for doc in results:
    print(doc)



Most Popular Course Categories:
{'totalEnrollments': 6, 'category': 'Web Dev'}
{'totalEnrollments': 6, 'category': 'Cloud'}
{'totalEnrollments': 4, 'category': 'Data Science'}


In [91]:
#Task 4.2.4 Student engagement
# Submissions per student
pipeline_engagement = [
    {
        "$group": {
            "_id": "$studentId",
            "submissions": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "studentId": "$_id",
            "submissions": 1
        }
    },
    {"$sort": {"submissions": -1}}
]

results = db.submissions.aggregate(pipeline_engagement)
print("\nStudent Engagement (Submissions per Student):")
for doc in results:
    print(doc)

# Completion rate (students who completed a course / total enrolled)
pipeline_completion_rate = [
    {
        "$group": {
            "_id": "$status",
            "count": {"$sum": 1}
        }
    }
]

results = list(db.enrollments.aggregate(pipeline_completion_rate))
total = sum(doc["count"] for doc in results)
completed = next((doc["count"] for doc in results if doc["_id"] == "completed"), 0)
completion_rate = (completed / total) * 100 if total else 0
print(f"\nOverall Course Completion Rate: {round(completion_rate, 2)}%")



Student Engagement (Submissions per Student):
{'submissions': 4, 'studentId': 'd3a329fe-12b8-4e19-917c-6ec05b1f148b'}
{'submissions': 3, 'studentId': '1158dfed-6077-492d-b037-f066f28256fc'}
{'submissions': 3, 'studentId': '2147e3e1-112f-4510-a013-f36e7ae1ea34'}
{'submissions': 2, 'studentId': '445f7800-3286-4c4f-9b64-def7566e8364'}
{'submissions': 2, 'studentId': '40e849aa-21d8-4174-8a9a-8f878982377f'}
{'submissions': 2, 'studentId': '46d6939a-3e62-442e-a352-c1a156807c5f'}
{'submissions': 2, 'studentId': '4bb9aa90-523c-4930-808e-612a205bbcb7'}
{'submissions': 2, 'studentId': '0f29b289-49b8-4fee-b744-4b70bb8f19cd'}
{'submissions': 1, 'studentId': '4d952d16-fbd1-4256-aef9-907de165da21'}
{'submissions': 1, 'studentId': '1cb53613-d99c-4033-bafe-fbd0c10ab630'}
{'submissions': 1, 'studentId': '0bd95c3e-a85c-4079-9de0-4adf45db6ec7'}
{'submissions': 1, 'studentId': '097a1659-9b60-4f2a-922a-dfe0fc6f70a0'}

Overall Course Completion Rate: 37.5%


In [92]:
#Task 5.1.1 Index creation: User Email LookUP
db.users.create_index("email", unique=True)

'email_1'

In [93]:
#Task 5.1.2 Course search by title and category
db.courses.create_index([("title", 1), ("category", 1)])

'title_1_category_1'

In [94]:
#Task 5.1.3 Assignment queries by due date
db.assignments.create_index("dueDate")

'dueDate_1'

In [95]:
#Task 5.1.4 enrollment Enrollment queries by student and course
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])


'studentId_1_courseId_1'

In [97]:
#Task 5.2.1 Query performance with explain()
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

# Example: Find a user by email
explanation = db.users.find({"email": "student@example.com"}).explain()
print(explanation)

{'queryPlanner': {'plannerVersion': 1, 'namespace': 'eduhub_db.users', 'indexFilterSet': False, 'parsedQuery': {'email': {'$eq': 'student@example.com'}}, 'winningPlan': {'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'email': 1}, 'indexName': 'email_1', 'isMultiKey': False, 'multiKeyPaths': {'email': []}, 'isUnique': True, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'email': ['["student@example.com", "student@example.com"]']}}}, 'rejectedPlans': []}, 'executionStats': {'executionSuccess': True, 'nReturned': 0, 'executionTimeMillis': 24, 'totalKeysExamined': 0, 'totalDocsExamined': 0, 'executionStages': {'stage': 'FETCH', 'nReturned': 0, 'executionTimeMillisEstimate': 22, 'works': 1, 'advanced': 0, 'needTime': 0, 'needYield': 0, 'saveState': 0, 'restoreState': 0, 'isEOF': 1, 'docsExamined': 0, 'alreadyHasObj': 0, 'inputStage': {'stage': 'IXSCAN', 'nReturned': 0, 'executionTimeMillisEstimate': 22, 'works': 1, 'adva

In [100]:
#Task 5.2.2 Optimize 3 slow queries, 1
import time
def time_query(label, query_func):
    start = time.time()
    result = query_func()
    end = time.time()
    duration = round(end - start, 4)
    print(f"{label} → {duration} seconds")
    return result

# Create index (if not yet created)
db.users.create_index("email", unique=True)

# Define the query function
def query_user_by_email():
    return list(db.users.find({"email": "student@example.com"}))

# Run with timer
time_query("User Email Lookup", query_user_by_email)


User Email Lookup → 0.003 seconds


[]

In [101]:
#Task 5.2.2 Optimize 3 slow queries, 2
db.courses.create_index([("title", "text"), ("category", "text")])

# Define the text search function
def search_courses_by_title():
    return list(db.courses.find({"$text": {"$search": "python"}}))

# Run with timer
time_query("Course Title Search", search_courses_by_title)


Course Title Search → 0.0291 seconds


[{'_id': ObjectId('684fd0c51e9483271544a656'),
  'courseId': 'bb2f6188-c83e-45e9-9091-523f57beb49c',
  'title': 'Data Visualization with Python',
  'description': 'Learn how to visualize data using matplotlib and seaborn.',
  'instructorId': '363b3454-b7e4-430b-813b-bd84fe263cb8',
  'category': 'Data Science',
  'level': 'intermediate',
  'duration': 5.5,
  'price': 89.99,
  'tags': ['data', 'visualization', 'python'],
  'createdAt': datetime.datetime(2025, 6, 16, 2, 7, 33, 21000),
  'updatedAt': datetime.datetime(2025, 6, 16, 2, 7, 33, 21000),
  'isPublished': True}]

In [102]:
#Task 5.2.2 Optimize 3 slow queries, 3
# Ensure index exists
db.assignments.create_index("dueDate")

# Define the time-based query
def upcoming_assignments():
    next_week = datetime.now() + timedelta(days=7)
    return list(db.assignments.find({"dueDate": {"$lte": next_week}}))

# Run with timer
time_query("Assignments Due in 7 Days", upcoming_assignments)


Assignments Due in 7 Days → 0.023 seconds


[]

In [103]:
#Task 6.1 Schema validation. 
#Ths handles missing email (required field),wrong role value(not in student  or instructor)
#This handles invalid email like test@com (rejection reason is if it fails regex)
#This handles skills not an array(rejection reason is TYPE mismatch)
#This handles dateJoined as string (rejection reason will be TYPE mismatch)

user_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["firstName", "lastName", "email", "role"],
        "properties": {
            "firstName": {
                "bsonType": "string",
                "description": "Must be a string and is required"
            },
            "lastName": {
                "bsonType": "string",
                "description": "Must be a string and is required"
            },
            "email": {
                "bsonType": "string",
                "pattern": "^\\S+@\\S+\\.\\S+$",
                "description": "Must be a valid email address"
            },
            "role": {
                "enum": ["student", "instructor"],
                "description": "Can only be 'student' or 'instructor'"
            },
            "dateJoined": {
                "bsonType": "date",
                "description": "Must be a date"
            },
            "profile": {
                "bsonType": "object",
                "properties": {
                    "bio": {"bsonType": "string"},
                    "avatar": {"bsonType": "string"},
                    "skills": {
                        "bsonType": "array",
                        "items": {"bsonType": "string"}
                    }
                }
            },
            "isActive": {
                "bsonType": "bool",
                "description": "Must be true or false"
            }
        }
    }
}


In [109]:
#Task 6.2
from pymongo import MongoClient, errors
from datetime import datetime

In [112]:
#Export all collections to sample_data.json
from pymongo import MongoClient
from bson.json_util import dumps
import json

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

# List all collections
collections = db.list_collection_names()

# Export data from each collection
export_data = {}

for coll_name in collections:
    data = list(db[coll_name].find())
    export_data[coll_name] = data

# Write to JSON file
with open("sample_data.json", "w") as f:
    f.write(dumps(export_data, indent=2))

print("Exported all collection data to 'sample_data.json'")


Exported all collection data to 'sample_data.json'


In [113]:
#Export of each collection to a folder "Sample_data_exports"
from pymongo import MongoClient
from bson.json_util import dumps
import os

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

# Create output directory (if not exists)
output_dir = "sample_data_exports"
os.makedirs(output_dir, exist_ok=True)

# List and export each collection
collections = db.list_collection_names()

for coll_name in collections:
    data = list(db[coll_name].find())
    filename = os.path.join(output_dir, f"{coll_name}.json")
    
    with open(filename, "w") as f:
        f.write(dumps(data, indent=2))
    
    print(f"Exported {coll_name} → {filename}")


Exported courses → sample_data_exports\courses.json
Exported assignments → sample_data_exports\assignments.json
Exported submissions → sample_data_exports\submissions.json
Exported users → sample_data_exports\users.json
Exported enrollments → sample_data_exports\enrollments.json
Exported lessons → sample_data_exports\lessons.json


In [114]:
from bson.json_util import dumps
import pandas as pd
import os

In [115]:
# Output directory, this will export to JSON and CSV format.
output_dir = "sample_data_exports"
os.makedirs(output_dir, exist_ok=True)

# Collections to export
collections = db.list_collection_names()

for coll_name in collections:
    # Fetch data
    data = list(db[coll_name].find())

    # Export to JSON
    json_path = os.path.join(output_dir, f"{coll_name}.json")
    with open(json_path, "w") as f:
        f.write(dumps(data, indent=2))
    
    # Export to CSV (flattening embedded fields)
    try:
        df = pd.json_normalize(data)
        csv_path = os.path.join(output_dir, f"{coll_name}.csv")
        df.to_csv(csv_path, index=False)
        print(f"✅ Exported {coll_name} to CSV and JSON.")
    except Exception as e:
        print(f"Could not export {coll_name} to CSV: {e}")

✅ Exported courses to CSV and JSON.
✅ Exported assignments to CSV and JSON.
✅ Exported submissions to CSV and JSON.
✅ Exported users to CSV and JSON.
✅ Exported enrollments to CSV and JSON.
✅ Exported lessons to CSV and JSON.


In [118]:
#Check current indexes on the courses collection
for idx in db.courses.list_indexes():
    print(idx)


SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('title', 1), ('category', 1)])), ('name', 'title_1_category_1')])
SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', 'title_text_category_text'), ('weights', SON([('category', 1), ('title', 1)])), ('default_language', 'english'), ('language_override', 'language'), ('textIndexVersion', 3)])


In [119]:
#Bonus1; Text Search for Course Content
search_term = "data science"

results = db.courses.find({
    "$text": {
        "$search": search_term
    }
})

for course in results:
    print(f"{course['title']} - {course.get('description', '')}")


Data Visualization with Python - Learn how to visualize data using matplotlib and seaborn.
Course 5 - This is course 5 description.
Course 3 - This is course 3 description.


In [121]:
#Bonus 3: Data archiving strategy for old enrollments
#A.This will look at enrolment  where completed is true, completeDate or enrolledAt is older than one year

from datetime import datetime, timedelta

cutoff_date = datetime.now() - timedelta(days=365)
criteria = {
    "completed": True,
    "completionDate": {"$lt": cutoff_date}
}



In [ ]:
#Archive data (Move-and-delete Process)
# Step 1: Find old enrollments
old_enrollments = list(db.enrollments.find(criteria))

# Step 2: Insert into archive collection
if old_enrollments:
    db.enrollments_archive.insert_many(old_enrollments)

    # Step 3: Remove from active collection
    ids = [e["_id"] for e in old_enrollments]
    db.enrollments.delete_many({"_id": {"$in": ids}})

    print(f"✅ Archived {len(old_enrollments)} old enrollments.")
else:
    print("📂 No enrollments to archive.")

db.enrollments_archive.create_index("studentId")
db.enrollments_archive.create_index("courseId")
db.enrollments_archive.create_index("completionDate")


In [128]:
!jupyter nbconvert --to script eduhub_mongodb_project.ipynb

[NbConvertApp] Converting notebook eduhub_mongodb_project.ipynb to script
[NbConvertApp] Writing 39512 bytes to eduhub_mongodb_project.py
